# Chapter 34: Email Messages

This notebook covers Python's `email.message.EmailMessage` class for constructing, inspecting, and serializing email messages. You will learn how to set headers, add body content, work with attachments, and convert messages to strings.

## Key Concepts
- **`EmailMessage`**: The modern API for building email messages
- **Headers**: Subject, From, To, and other metadata fields
- **Body content**: Setting plain text and HTML content via `set_content()`
- **Attachments**: Adding binary and text attachments to messages
- **Serialization**: Converting messages to strings with `as_string()`
- **Multipart**: Messages with both body and attachments

## Section 1: Creating a Basic Email

An `EmailMessage` object represents a complete email. You create it, then set headers and body content individually.

In [ ]:
from email.message import EmailMessage

# Create a new email message
msg: EmailMessage = EmailMessage()

# Set standard headers
msg["Subject"] = "Test"
msg["From"] = "alice@example.com"
msg["To"] = "bob@example.com"

# Set the body content
msg.set_content("Hello, Bob!")

print(f"Subject: {msg['Subject']}")
print(f"From:    {msg['From']}")
print(f"To:      {msg['To']}")
print(f"Body:    {msg.get_content()!r}")

In [ ]:
from email.message import EmailMessage

# Verify header values match what was set
msg: EmailMessage = EmailMessage()
msg["Subject"] = "Test"
msg["From"] = "alice@example.com"
msg["To"] = "bob@example.com"
msg.set_content("Hello, Bob!")

assert msg["Subject"] == "Test"
assert msg["From"] == "alice@example.com"
assert "Hello, Bob!" in msg.get_content()

print("All assertions passed.")

## Section 2: Working with Headers

Email headers are case-insensitive. You can access them using dictionary-style syntax, iterate over them, and check for existence.

In [ ]:
from email.message import EmailMessage

# Headers are case-insensitive
msg: EmailMessage = EmailMessage()
msg["Content-Type"] = "text/plain"

# Access with different casing
print(f"msg['Content-Type']:  {msg['Content-Type']}")
print(f"msg['content-type']:  {msg['content-type']}")
print(f"msg['CONTENT-TYPE']:  {msg['CONTENT-TYPE']}")

assert msg["content-type"] == "text/plain"
print("\nCase-insensitive access confirmed.")

In [ ]:
from email.message import EmailMessage

# Exploring all headers on a message
msg: EmailMessage = EmailMessage()
msg["Subject"] = "Weekly Report"
msg["From"] = "reports@example.com"
msg["To"] = "team@example.com"
msg["CC"] = "manager@example.com"
msg["Date"] = "Fri, 21 Feb 2026 10:00:00 +0000"
msg.set_content("Please find the weekly report attached.")

# Iterate over all header keys
print("All headers:")
for key in msg.keys():
    print(f"  {key}: {msg[key]}")

# Check if a header exists
print(f"\nHas 'CC':     {'CC' in msg}")
print(f"Has 'BCC':    {'BCC' in msg}")

In [ ]:
from email.message import EmailMessage

# Replacing and deleting headers
msg: EmailMessage = EmailMessage()
msg["Subject"] = "Draft"
print(f"Before: {msg['Subject']}")

# To replace a header, delete it first, then set the new value
del msg["Subject"]
msg["Subject"] = "Final Version"
print(f"After:  {msg['Subject']}")

# replace_header() is a convenient alternative
msg.replace_header("Subject", "Published")
print(f"Replaced: {msg['Subject']}")

## Section 3: Setting Body Content

The `set_content()` method sets the message body. By default it creates a `text/plain` message, but you can also set HTML content or specify other subtypes.

In [ ]:
from email.message import EmailMessage

# Plain text content (default)
msg: EmailMessage = EmailMessage()
msg["Subject"] = "Plain Text"
msg.set_content("This is a plain text email body.")

print(f"Content type: {msg.get_content_type()}")
print(f"Body: {msg.get_content()!r}")

In [ ]:
from email.message import EmailMessage

# HTML content
msg: EmailMessage = EmailMessage()
msg["Subject"] = "HTML Email"
msg.set_content(
    "<h1>Hello</h1><p>This is <b>HTML</b> content.</p>",
    subtype="html",
)

print(f"Content type: {msg.get_content_type()}")
print(f"Body preview: {msg.get_content()[:60]}...")

## Section 4: Attachments and Multipart Messages

Adding an attachment to a message makes it multipart. The original body becomes one part, and each attachment becomes another.

In [ ]:
from email.message import EmailMessage

# Create a message with an attachment
msg: EmailMessage = EmailMessage()
msg["Subject"] = "With attachment"
msg.set_content("Main body")

# Add a binary attachment
msg.add_attachment(
    b"file content",
    maintype="application",
    subtype="octet-stream",
    filename="data.bin",
)

print(f"Is multipart: {msg.is_multipart()}")
assert msg.is_multipart()

# Iterate over the parts
for i, part in enumerate(msg.iter_parts()):
    print(f"\nPart {i}:")
    print(f"  Content type: {part.get_content_type()}")
    filename: str | None = part.get_filename()
    if filename:
        print(f"  Filename:     {filename}")

In [ ]:
from email.message import EmailMessage

# Multiple attachments
msg: EmailMessage = EmailMessage()
msg["Subject"] = "Multiple Attachments"
msg.set_content("See attached files.")

# Attach a text file
msg.add_attachment(
    "This is a text file.",
    subtype="plain",
    filename="notes.txt",
)

# Attach binary data
msg.add_attachment(
    b"\x89PNG\r\n\x1a\n",
    maintype="image",
    subtype="png",
    filename="logo.png",
)

parts: list[str] = []
for part in msg.iter_parts():
    ct: str = part.get_content_type()
    fn: str | None = part.get_filename()
    parts.append(f"{ct} ({fn or 'body'})")

print("Parts:")
for p in parts:
    print(f"  {p}")

## Section 5: Serializing Messages to Strings

The `as_string()` method produces the full RFC 2822 representation of the message, including headers and encoded body. This is the format used when actually sending an email.

In [ ]:
from email.message import EmailMessage

# Serialize a simple message
msg: EmailMessage = EmailMessage()
msg["Subject"] = "Hello"
msg.set_content("Body text")

text: str = msg.as_string()

# The serialized form contains headers and body
assert "Subject: Hello" in text
assert "Body text" in text

print("Serialized message:")
print(text)

In [ ]:
from email.message import EmailMessage

# as_bytes() produces bytes instead of str
msg: EmailMessage = EmailMessage()
msg["Subject"] = "Binary form"
msg.set_content("Hello in bytes.")

raw: bytes = msg.as_bytes()
print(f"Type: {type(raw).__name__}")
print(f"First 80 bytes: {raw[:80]}")

## Section 6: Parsing Email Strings

The `email.message_from_string()` function parses a raw email string back into an `EmailMessage` (or `Message`) object. The `email.policy.default` policy returns the modern `EmailMessage` type.

In [ ]:
from email import message_from_string
from email.message import EmailMessage
from email.policy import default

# Build a raw email string
raw_email: str = (
    "Subject: Parsed Email\n"
    "From: sender@example.com\n"
    "To: receiver@example.com\n"
    "Content-Type: text/plain; charset=\"utf-8\"\n"
    "\n"
    "This is the body of a parsed email.\n"
)

# Parse with the default policy for EmailMessage
parsed: EmailMessage = message_from_string(raw_email, policy=default)

print(f"Type:    {type(parsed).__name__}")
print(f"Subject: {parsed['Subject']}")
print(f"From:    {parsed['From']}")
print(f"Body:    {parsed.get_content()!r}")

## Section 7: Content Type and Encoding Inspection

`EmailMessage` provides methods to inspect the content type, character set, and transfer encoding of the message.

In [ ]:
from email.message import EmailMessage

# Inspect content metadata
msg: EmailMessage = EmailMessage()
msg.set_content("Unicode: \u00e9\u00e0\u00fc")

print(f"Content type:     {msg.get_content_type()}")
print(f"Main type:        {msg.get_content_maintype()}")
print(f"Sub type:         {msg.get_content_subtype()}")
print(f"Charset:          {msg.get_param('charset')}")
print(f"Content-Encoding: {msg['Content-Transfer-Encoding']}")

## Section 8: Practical Patterns

Common patterns for constructing email messages in real applications.

In [ ]:
from email.message import EmailMessage


def build_email(
    subject: str,
    from_addr: str,
    to_addr: str,
    body: str,
    attachments: list[tuple[bytes, str, str, str]] | None = None,
) -> EmailMessage:
    """Build an email message with optional attachments.

    Each attachment is (data, maintype, subtype, filename).
    """
    msg: EmailMessage = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = from_addr
    msg["To"] = to_addr
    msg.set_content(body)

    if attachments:
        for data, maintype, subtype, filename in attachments:
            msg.add_attachment(
                data,
                maintype=maintype,
                subtype=subtype,
                filename=filename,
            )

    return msg


# Build a message with attachments
email: EmailMessage = build_email(
    subject="Project Update",
    from_addr="dev@example.com",
    to_addr="pm@example.com",
    body="Here are the latest build artifacts.",
    attachments=[
        (b"build log data", "text", "plain", "build.log"),
        (b"\x00\x01\x02", "application", "octet-stream", "artifact.bin"),
    ],
)

print(f"Subject:     {email['Subject']}")
print(f"Multipart:   {email.is_multipart()}")
print(f"Part count:  {len(list(email.iter_parts()))}")

In [ ]:
from email.message import EmailMessage


def extract_attachments(msg: EmailMessage) -> list[tuple[str, int]]:
    """Extract attachment filenames and sizes from a message."""
    results: list[tuple[str, int]] = []
    if not msg.is_multipart():
        return results

    for part in msg.iter_attachments():
        filename: str = part.get_filename() or "unnamed"
        content: bytes | str = part.get_content()
        size: int = len(content) if isinstance(content, (bytes, str)) else 0
        results.append((filename, size))
    return results


# Create a message with attachments
msg: EmailMessage = EmailMessage()
msg.set_content("Body")
msg.add_attachment(b"A" * 100, maintype="application", subtype="octet-stream", filename="big.bin")
msg.add_attachment(b"small", maintype="application", subtype="octet-stream", filename="small.bin")

for name, size in extract_attachments(msg):
    print(f"  {name}: {size} bytes")

## Summary

### Core Class
- **`EmailMessage()`**: Create a new email message object

### Setting Content
- **`msg['Header'] = value`**: Set a header (Subject, From, To, etc.)
- **`msg.set_content(text)`**: Set the body as plain text (default) or HTML (`subtype="html"`)
- **`msg.add_attachment(data, maintype, subtype, filename)`**: Attach a file

### Reading Content
- **`msg['Header']`**: Read a header (case-insensitive)
- **`msg.get_content()`**: Get the body content
- **`msg.get_content_type()`**: Get the MIME type (e.g., `text/plain`)
- **`msg.is_multipart()`**: Check if the message has multiple parts
- **`msg.iter_parts()`**: Iterate over all parts of a multipart message
- **`msg.iter_attachments()`**: Iterate over attachment parts only

### Serialization
- **`msg.as_string()`**: Serialize to a string (RFC 2822 format)
- **`msg.as_bytes()`**: Serialize to bytes
- **`message_from_string(text, policy=default)`**: Parse a raw email string

### Important Notes
- Headers are **case-insensitive** when accessed
- Adding an attachment automatically makes the message **multipart**
- Use `del msg['Header']` followed by assignment to **replace** a header, or use `replace_header()`
- Use `policy=email.policy.default` when parsing to get `EmailMessage` objects